In [ ]:
import pandas as pd
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.io.shapereader as shpreader
import shapely.geometry as sgeom

hv.extension('bokeh')

## Define data

In [ ]:
def sample_data():
    """
    Returns a list of latitudes and a list of longitudes (lons, lats)
    for Hurricane Katrina (2005).

    The data was originally sourced from the HURDAT2 dataset from AOML/NOAA:
    http://www.aoml.noaa.gov/hrd/hurdat/newhurdat-all.html on 14th Dec 2012.

    """
    lons = [-75.1, -75.7, -76.2, -76.5, -76.9, -77.7, -78.4, -79.0,
            -79.6, -80.1, -80.3, -81.3, -82.0, -82.6, -83.3, -84.0,
            -84.7, -85.3, -85.9, -86.7, -87.7, -88.6, -89.2, -89.6,
            -89.6, -89.6, -89.6, -89.6, -89.1, -88.6, -88.0, -87.0,
            -85.3, -82.9]

    lats = [23.1, 23.4, 23.8, 24.5, 25.4, 26.0, 26.1, 26.2, 26.2, 26.0,
            25.9, 25.4, 25.1, 24.9, 24.6, 24.4, 24.4, 24.5, 24.8, 25.2,
            25.7, 26.3, 27.2, 28.2, 29.3, 29.5, 30.2, 31.1, 32.6, 34.1,
            35.6, 37.0, 38.6, 40.1]

    return lons, lats

shapename = 'admin_1_states_provinces_lakes_shp'
states_shp = shpreader.natural_earth(resolution='110m',
                                     category='cultural', name=shapename)

lons, lats = sample_data()
track = sgeom.LineString(zip(lons, lats))

title = 'US States which intersect the track of Hurricane Katrina (2005)'

track_buffer = track.buffer(2)
shapes = []
for state in shpreader.Reader(states_shp).geometries():
    # pick a default color for the land with a black outline,
    # this will change if the storm intersects with our track
    facecolor = (0.9375, 0.9375, 0.859375)

    if state.intersects(track):
        facecolor = 'red'
    elif state.intersects(track_buffer):
        facecolor = '#FF7E00'
    shapes.append(gv.Shape(state).opts(style=dict(fill_color=facecolor)))
shapes.append(gv.Shape(track_buffer).opts(style=dict(alpha=0.5)))
shapes.append(gv.Shape(track))

## Plot

In [ ]:
%%opts Shape [width=700 height=500] Polygons Path [show_legend=True apply_ranges=False]
# Add proxy artists for legend
direct_hit = hv.Polygons([[(0,0)]], label='State directly intersects\nwith track').opts(style=dict(color='red'))
within_2_deg = hv.Polygons([[(0,0)]], label='State is within \n2 degrees of track').opts(style=dict(color='#FF7E00'))
hv.Overlay(shapes).relabel(title) * direct_hit * within_2_deg